In [15]:
import numpy as np
import os
import torch

In [35]:
data = np.load("/home/hansika/gem5/gem5/scripts/numpy_data_test/64_nodes/X/2.npy",  allow_pickle=True)

In [41]:
data.size/900

76608.0

In [1]:
np.count_nonzero(data == 1)

NameError: name 'np' is not defined

In [92]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data_dir, file_index,):
        self.data_dir = data_dir
        self.file_index = file_index
        y = np.load(self.data_dir + "Y/" + str(self.file_index) + ".npy", allow_pickle=True)
        self.y = torch.from_numpy(y)
    
    def __getitem__(self, index):
        x = np.load(self.data_dir + "X/" + str(self.file_index) + ".npy", allow_pickle=True, mmap_mode='r')
        x = torch.from_numpy(x)
        return [x[index], self.y[index]]
    
    def __len__(self):
        return len(self.y)

In [95]:
data = MyDataset("/home/hansika/gem5/gem5/scripts/numpy_data/64_nodes/",0)

In [96]:
len(data)


399168

In [98]:
data[1][0]

tensor([[  7,  50,  52,   1,   1,   1,   2,  31,  94,  60,   1,   1,   1,   2,
           4,   1,   1,   1,   2, 176, 156,  29,  44, 190,  21,   1,   1,   1,
           2,  10,   1,   1,   1,   2,  58,  57,  35,  24,   1,   1,   1,   2,
          27,   1,   1,   1,   2,  69,   1,   1,   1,   2,   3,   1,   1,   1,
           2,  22,  42,   1,   1,   1,   2,  68,  77,   8,   1,   1,   1,   2,
         172,   1,   1,   1,   2,  60,   1,   1,   1,   2,  35,   1,   1,   1,
           2,  19,   1,   1,   1,   2, 140,  78, 151, 333, 102,   1,   1,   1,
           1,   1,  66,  22,   1,   1,   1,   2,  38,  34, 123,   1,   1,   1,
           2,  26, 196,  76,   1,   1,   1,   2,   1,   1,   1,   1,   1,  68,
           1,   1,   1,   2,  54,   1,   1,   1,   2,  80,   1,   1,   1,   2,
          48,   1,   1,   1,   2,   1,   7, 127,  64,   1,   1,   1,   2, 134,
           1,   1,   1,   2,  46,   1,   1,   1,   2,  34, 140,   1,   1,   1,
           2,  19,  92,   3, 127,  65, 180,   1,   1

In [103]:
list_of_dataset = []
number_of_files = 41
from torch.utils.data import ConcatDataset

for i in range(number_of_files):
    list_of_dataset.append(MyDataset("/home/hansika/gem5/gem5/scripts/numpy_data/64_nodes/",i))

full_dataset = ConcatDataset(list_of_dataset)


    

In [104]:
len(full_dataset)

16257024

In [106]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# W1, W2, K1, K2 are hyper parameters that eventually needed training
W1 = 30
W2 = 10
K1= 2000
K2 = 1000

#dummy data to try the NN ( 2 arrays of size 500)
dummy = torch.randn(2,500).view(-1,1,2,500)

# represents the whole CNN
class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, K1, (2,W1), stride=(2, 1))
        self.pool1 = nn.MaxPool2d((1, 5), stride=(1, 1))
        self.conv2 = nn.Conv2d(K1, K2, (1,W2), stride=(1, 1))
        self.pool2 = nn.MaxPool2d((1, 5), stride=(1, 1))
        
        self.fc1 = nn.Linear(1000*454, 3000) # need to automate arriving at this number (1000*254)
        self.fc2 = nn.Linear(3000, 800) 
        self.fc3 = nn.Linear(800,100)
        self.fc4 = nn.Linear(100,1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        x = x.view(-1, 1000*454)    
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return torch.sigmoid(x)
        

    
## ------------------------------------testing small net--------------------------- ##

# net = SmallNet()
# print(net)
# print(type(dummy))

# output = net.forward(dummy.narrow(2,0,2))
# output.shape

## --------------------------------end testing small net----------------------------- ##

## ---------------------------------------- testing net ----------------------------- ##

net = Net()
print(net)

output = net.forward(dummy)
output.shape

## -------------------------------end testing net------------------------------------ ##



# ------------------- Training the CNN ------------------------------------- ##
# For now this code is only to show the structure, I need to add data preparation and modify code accordingly.

isTraining = False
if isTraining:
    
    # need to use collected data here
    trainset = []
    testset = []


    BATCH_SIZE = 8
    EPOCHS = 3

    # learning rate of the adam optimizer should be a hyperparameter
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(EPOCHS):
        for data in trainset:
            X, y = data 
            net.zero_grad()  
            output = net(X.view(-1,784))  
            loss = nn.CrossEntropyLoss()
            loss = loss(input, y)
            loss.backward() 
            optimizer.step()  
        print(loss)  


    correct = 0
    total = 0

    with torch.no_grad():
        for data in testset:
            X, y = data
            output = net(X.view(-1,784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total += 1

    print("Accuracy: ", round(correct/total, 3))    

Net(
  (conv1): Conv2d(1, 2000, kernel_size=(2, 30), stride=(2, 1))
  (pool1): MaxPool2d(kernel_size=(1, 5), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(2000, 1000, kernel_size=(1, 10), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(1, 5), stride=(1, 1), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=454000, out_features=3000, bias=True)
  (fc2): Linear(in_features=3000, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=100, bias=True)
  (fc4): Linear(in_features=100, out_features=1, bias=True)
)


/home/hansika/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [108]:
output

tensor([[0.5096]], grad_fn=<SigmoidBackward>)